In [1]:
from bs4 import BeautifulSoup as BS
import urllib as url
import pandas as pd
import numpy as np

In [2]:
# Initializing Request object by passing in the earthquake's url, and assigning the reults to resulttext
request = url.request.Request("https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900")
result = url.request.urlopen(request)
resulttext = result.read()

In [4]:
# Initialize a BeautifulSoup instance of the sring-ed version of the website's html
soup = BS(resulttext, 'html.parser')

In [5]:
# There are six different tables on the website, so I am targetting the table with specific classes. 
# This returns a list, so I assert that the list length is equal to one (so I know I am only working with one table),
# then assign the first index to my variable to reference.
earthquake_table_list = soup.find_all('table', class_=['sortable', 'wikitable', 'jquery-tablesorter'])
assert len(earthquake_table_list) == 1

earthquake_table = earthquake_table_list[0]

In [6]:
# Grab all of the 'th' elements, grab the text from each one, and assign the results to a list. 
# This will be used when creating the pandas df for column name assignment
table_header = earthquake_table.find_all('th')

columns = [th.text for th in table_header]

In [7]:
print(columns)

['Origin (UTC)', 'Present-day country and link to Wikipedia article', 'Lat', 'Long', 'Depth (km)', 'Magnitude', 'Secondary Effects', 'PDE Shaking Deaths', 'PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', 'Other Source Deaths']


In [8]:
# Find all table rows
table_rows = earthquake_table.find_all('tr')[1:]

# Create an empty list that will ultimately hold.. more lists. The nested lists will hold each individual value per 
# column, per row
df_rows = list()
for row in table_rows:
    # Grab the text from each row, and assign it to a list. Append this list to df_rows
    row_data= [data.text for data in row.find_all('td')]
    
    df_rows.append(row_data)

In [9]:
for row in df_rows[:5]:
    print(row)

['1900-05-11 17:23', 'Japan', '38.700', '141.100', '5', '7.0 MJMA', '', '', '', '', '', '']
['1900-07-12 06:25', 'Turkey', '40.300', '43.100', '', '5.9 Muk', '', '', '', '140', '', '']
['1900-10-29 09:11', 'Venezuela', '11.000', '-66.000', '0', '7.7 Mw', '', '', '', '', '', '']
['1901-02-15 00:00', 'China', '26.000', '100.100', '0', '6.5 Ms', '', '', '', '', '', '']
['1901-03-31 07:11', 'Bulgaria', '43.400', '28.700', '', '6.4 Muk', '', '', '', '4', '', '']


In [31]:
# Create a pandas df using the columns and rows we scraped.
earthquake_df = pd.DataFrame(data=df_rows, columns=columns)

# Since the 'Origin (UTC)' column is a date string, it would be 
earthquake_df['Origin (UTC)'] = pd.to_datetime(earthquake_df['Origin (UTC)'])
earthquake_df.set_index('Origin (UTC)', inplace=True)

earthquake_df.replace(r'^\s*$', np.nan, regex=True, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 12 columns):
Origin (UTC)                                         1340 non-null object
Present-day country and link to Wikipedia article    1340 non-null object
Lat                                                  1340 non-null object
Long                                                 1340 non-null object
Depth (km)                                           1340 non-null object
Magnitude                                            1340 non-null object
Secondary Effects                                    1340 non-null object
PDE Shaking Deaths                                   1340 non-null object
PDE Total Deaths                                     1340 non-null object
Utsu Total Deaths                                    1340 non-null object
EM-DAT Total Deaths                                  1340 non-null object
Other Source Deaths                                  1338 non-null object
dtype

In [11]:
earthquake_df.head()

,Present-day country and link to Wikipedia article,Lat,Long,Depth (km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths,Other Source Deaths
Origin (UTC),,,,,,,,,,,
1900-05-11 17:23:00,Japan,38.700,141.100,5,7.0 MJMA,NaN,NaN,NaN,NaN,NaN,NaN
1900-07-12 06:25:00,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,NaN,140,NaN,NaN
1900-10-29 09:11:00,Venezuela,11.000,-66.000,0,7.7 Mw,NaN,NaN,NaN,NaN,NaN,NaN
1901-02-15 00:00:00,China,26.000,100.100,0,6.5 Ms,NaN,NaN,NaN,NaN,NaN,NaN
1901-03-31 07:11:00,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,NaN,4,NaN,NaN


In [12]:
earthquake_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1340 entries, 1900-05-11 17:23:00 to 2011-09-23 10:41:00
Data columns (total 11 columns):
Present-day country and link to Wikipedia article    1340 non-null object
Lat                                                  1326 non-null object
Long                                                 1326 non-null object
Depth (km)                                           1250 non-null object
Magnitude                                            1339 non-null object
Secondary Effects                                    373 non-null object
PDE Shaking Deaths                                   739 non-null object
PDE Total Deaths                                     750 non-null object
Utsu Total Deaths                                    1027 non-null object
EM-DAT Total Deaths                                  560 non-null object
Other Source Deaths                                  37 non-null object
dtypes: object(11)
memory usage: 125.6+ KB
